In [71]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

In [72]:
# Importamos Librerias
import numpy as np
from collections import Counter
from sortedcontainers import SortedDict
import sklearn.naive_bayes as nb
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict

In [73]:
class Datos:

  TiposDeAtributos=('Continuo','Nominal')

  # TODO: procesar el fichero para asignar correctamente las variables tipoAtributos, nombreAtributos, nominalAtributos, datos y diccionarios
  # NOTA: No confundir TiposDeAtributos con tipoAtributos
  def __init__(self, nombreFichero):

      with open(nombreFichero, "r") as f:
        # Guardamos el numero de datos que contiene el DataSet y esta en la primera linea
        self.numDatos = int(f.readline())

        # Guardamos el nombre de los atributos
        self.nombreAtributos = f.readline().strip('\n').split(',')
        #print(self.nombreAtributos)

        # Leemos el tipo de los atributos de las variables y eliminamos el ultimo que es un salto de linea
        self.tipoAtributos = f.readline().strip('\n').split(',')
        #print(self.tipoAtributos)

        # Comprobamos que todos los atributos sean Continuos o Nominales
        if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
            raise ValueError("Tipo de atributo erroneo")

        # Segun el atributo, asignamos True o False.
        self.nominalAtributos = []

        # Guardamos en la lista nominalAtributos en la posicion de cada uno si es o no Nominal
        for tipo in self.tipoAtributos:
            if tipo == self.TiposDeAtributos[0]:
                self.nominalAtributos.append(False)
            else:
                self.nominalAtributos.append(True)
        #print(self.nominalAtributos)

        # Guardamos los datos del fichero y los formateamos, de tal forma que cada linea es una lista
        datos = f.readlines()
        datosFormat = []
        for lista in datos:
            datosFormat.append(lista.strip('\n').split(','))

        # print(set(sorted(datosFormat[0])))
        listaDatosAtributos = []
        for i in range(len(self.tipoAtributos)):
            listaDatosAtributos.append([])

        # Hacemos la traspuesta de los datos que guardamos para que cada lista de atributo guarde todos los datos
        # de cada atributo.
        for lista in datosFormat:
            i = 0
            for item in lista:
                listaDatosAtributos[i].append(item)
                i += 1

        # Ordenamos y hacemos un set para eliminar repetidos.
        i = 0
        for item in listaDatosAtributos:
            listaDatosAtributos[i] = sorted(set(item))
            i += 1


        # Creacion de lista diccionarios, en caso de que el atributo sea Continuo, el diccionario estara vacio
        self.listaDicts = []
        for i in range(len(self.tipoAtributos)):
            self.listaDicts.append({})

        # Creamos el diccionario tal y como se describe en las diapositivas, por orden y asignando valores numericos crecientes
        i = 0
        for atributo in listaDatosAtributos:
            k = 0
            if self.tipoAtributos[i] == "Nominal":
                for dato in atributo:
                    self.listaDicts[i][dato] = k
                    k += 1
            i += 1

        # Creacion de la matriz de datos utilizando el diccionario para mapear los valores
        # En primer lugar, creamos una matriz vacia de tamaña numero de atributos.
        self.datos = np.empty((int(self.numDatos),int(len(self.tipoAtributos))))
        i = 0
        j = 0

        # Metemos los datos en la matriz, mapeando con los diccionarios en el caso de que sean Nominales, y si son continuos normal.
        for i in range(int(self.numDatos)):
            for j in range(len(self.tipoAtributos)):
                if self.tipoAtributos[j] == 'Nominal':
                    self.datos[i][j] = self.listaDicts[j].get(str(datosFormat[i][j]))
                else:
                    self.datos[i][j] = datosFormat[i][j]
        
        print(self.nombreAtributos)
        print(self.listaDicts)
        f.close()


  # TODO: implementar en la practica 1
  def extraeDatos(self, idx):
    return self.datos[idx]


In [74]:
dataset = Datos('lenses.data')

print(dataset.datos)

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]


In [75]:
from abc import ABCMeta,abstractmethod


class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self,train=[],test=[]):
        self.indicesTrain=train
        self.indicesTest=test
    
    def __str__(self):
        return "Train: {}\nTest:  {}".format(str(self.indicesTrain),str(self.indicesTest)) 

#####################################################################################################

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Lista de las particiones
    def __init__(self, nombre=""):
        self.nombreEstrategia = nombre
        self.numeroParticiones = 0
        self.particiones=[]

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self,datos,seed=None):
        pass

In [76]:
class ValidacionSimple(EstrategiaParticionado):
    
    def __init__(self, porcentaje):
        self.porcentaje = porcentaje
        super().__init__("Validacion simple")
        
  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)

        self.numeroParticiones = 1
    
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Creamos la particion, en funcion del porcentaje especificado
        self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos*self.porcentaje)],
                                      indicesAleatorios[int(datos.numDatos*self.porcentaje):])]
        
        return self.particiones
    

In [77]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
print(validacion_simple.particiones[0])

Train: [ 4  6 23  9 14  5 13 15  7 10  1 21 20 22 16 11 18  3]
Test:  [17  0 12  8  2 19]


In [78]:
class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    
    def __init__(self, k):
        self.k = k
        super().__init__("Validacion cruzada")

    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)
        
        self.numeroParticiones = self.k
        
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Hallamos el tamaño de cada bloque
        tamBloque = int(datos.numDatos/self.k)
        
        
        datosSobran = datos.numDatos - (tamBloque*self.k)
        count = 0
        for i in range(self.k):
            
            train = np.delete(indicesAleatorios, range(i*tamBloque,(i+1)*tamBloque))
            test =  indicesAleatorios[i*tamBloque:(i+1)*tamBloque]
            
            # Caso en el que la cuenta es justa
            if datosSobran == 0:
                self.particiones.append(Particion(train, test))
                
            # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
            if datosSobran > 0:
                count += 1
                particionTest = np.append(test, train[(datos.numDatos - tamBloque)- i - 1])
                particionTrain = np.delete(train, (datos.numDatos - tamBloque)- i - 1)
                datosSobran -= 1
                self.particiones.append(Particion(particionTrain, particionTest))
                
            
           
                

In [79]:
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

for particion in v_cruzada.particiones:
    print(particion)

Train: [13  8  9 12 17 19 22  2 21  0 18 23 14  7 20  4  1  6]
Test:  [16 15 11  3 10  5]
Train: [16 15 11  3 10  5 22  2 21  0 18 23 14  7 20  4  1  6]
Test:  [13  8  9 12 17 19]
Train: [16 15 11  3 10  5 13  8  9 12 17 19 14  7 20  4  1  6]
Test:  [22  2 21  0 18 23]
Train: [16 15 11  3 10  5 13  8  9 12 17 19 22  2 21  0 18 23]
Test:  [14  7 20  4  1  6]


In [80]:
class Clasificador:
  
    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datos,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
        i = 0
        real = datos[:,-1]
        error = 0
        for i in range(len(real)):
            if real[i] != pred[i]:
                error += 1
        err = (error)/(len(real)+0.0)
        return err


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        errores = 0
        #particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones
    # Validación Simple
        if len(particionado.particiones) == 1:
            clasificador.entrenamiento(dataset, particionado.particiones[0].indicesTrain)
            pred = clasificador.clasifica(dataset,particionado.particiones[0].indicesTest)
            ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
            if ret > 0:
                return ret
            else:
                return 0
            
    # Validación Cruzada        
        else:
            for particion in particionado.particiones:
                clasificador.entrenamiento(dataset, particion.indicesTrain)
                pred = clasificador.clasifica(dataset,particion.indicesTest)
                ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
                errores += ret
            error = errores/len(particionado.particiones)
            #Devolucion de la media de los errores
            return error

In [115]:
class ClasificadorNaiveBayes(Clasificador):
    
    def __init__(self, laplace):
        self.laplace = laplace
        
    
    def entrenamiento(self,dataset,datosTrain):
     
        # Cargamos todos los datos de la clase del dataset desde la matriz de datos
        clasesTrain = dataset.extraeDatos(datosTrain)
        self.numClases = clasesTrain[:,-1]
        # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
        counter = Counter(self.numClases)
        # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero 
        # correspondiente a cada clase asignado en el diccionario
        self.dictPrioris={}
        for k in counter:
            k = int(k)
            counter[k] = counter[k]/len(self.numClases)
            self.dictPrioris[k] = counter[k]
            
        # Aqui ordenamos el diccionario para que esten en el mismo orden de como extraemos los datos del dataset
        self.dictPrioris = SortedDict(self.dictPrioris)
        
        # Calcular tablas de probabilidades del entrenamiento. Tenemos que calcular por cada atributo una cuenta
        # de las apariciones en cada clase
        # Creamos una lista de matrices, donde vamos almacenar todos los datos que hemos obtenido en los datos de Test
        self.posteriori = np.zeros(len(dataset.nombreAtributos)-1,dtype=object)
        
        # Recorremos todos los datos de la matriz sin llegar a la clase
        for i in range(len(dataset.nombreAtributos) - 1):
            
            # Si el dato que obtenemos es Nominal haremos el recuento de todas las veces que sale la P(D|H)
            if dataset.nominalAtributos[i] == True:
                
                # Creamos una matriz de tamaño X: Número de Atributos menos la clase Y: Número de clases
                post = np.zeros((len(dataset.listaDicts[i]),len(dataset.listaDicts[-1])))
               
                # Aqui contamos todos las datos que queremos del datos Train para construir la matriz de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    repes = Counter(dat[datosEnt[:,-1] == c])
                    for r in repes:
                        post[int(r),c] = repes[r]
                    if self.laplace == True:
                        self.posteriori[i] = post +1
                    else:
                        self.posteriori[i] = post
            
            # Si el dato es Continuo obtendremos la media y la desviación tipica de la clase
            else:
                
                # Creamos una matriz de X: Los datos de Media y Desivación típica Y: Número de clases
                post = np.zeros((2,len(dataset.listaDicts[-1])))
               
                # Aqui obtenemos la media y desviación tipica de cada clase, despues de tener los datos de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    datos = dat[datosEnt[:,-1] == c]
                    post[0][c] = np.mean(datos)
                    post[1][c] = np.std(datos)
                self.posteriori[i] = post
            
        #Calculamos los valores de los posteriori de todos las tablas anteriores
        for j in range(len(dataset.nominalAtributos)-1):
            if dataset.nominalAtributos[j] == True:
                for i in range(len(dataset.listaDicts) - 1):
                    self.posteriori[i] /= sum(self.posteriori[i])

        
    def clasifica(self,dataset,datosTest):
        j = 0
        aux = 1
        self.prediccion = []
        datTest = dataset.extraeDatos(datosTest)
        #Ahora vamos a estudiar la probabilidad de la clase con los datos obtenidos en el entrenamiento
        #Recorremos todos las datos de la matriz de los datos Test
        
        print(self.posteriori)
        for dato in datTest:
            mapa = []
            if dataset.nominalAtributos[j] == True:
            #Aqui obtenemos los prioris de cada clase para poder obtener la probabilidad de cada una
                for clase in range(len(self.dictPrioris)):
                #Aqui obtenemos cada valor posteriori de nuestro entrenamiento de los datos, es decir, P(D|H)
                    for atributo in range(len(self.posteriori)):
                        aux *= self.posteriori[atributo][int(dato[atributo])][clase]
                    
                    #Aqui obtenemos la siguiente probabilidad P(D|H)*P(H)
                    aux2 = self.dictPrioris.get(clase)*aux
                    aux = 1
                
                    #Lo añadimos a una lista para obtener la probabilidad de las diferentes clases
                    mapa.append(aux2)
            
            #Aqui obtenemos la probabilidad de los atibutos continuos
            else:
                for clase in range(len(self.dictPrioris)):
                    for atributo in range(len(self.posteriori)):
                        # Hacemos la formula de la distribucion normal
                        exp1 = 1/(self.posteriori[atributo][0][clase]*math.sqrt(2*math.pi))
                        exp2 = ((dato-self.posteriori[atributo][0][clase]) - dato[atributo]/self.posteriori[atributo][1][clase])
                        exp3 = exp2 ** 2
                        exp4 = math.exp((-1/2)* exp3)
                        total = exp1 * exp4
                    aux2 = total * self.dictPrioris.get(clase)
                    mapa.append(aux2)
            j +=1
            if j == len(dataset.nominalAtributos)-1:
                j = 0
            
            #Aqui obtenemos la predicción de mayor probabilidad y la guardamos en nuestra lista de predicciones
            self.prediccion.append(np.argmax(mapa))
        
        #Devolvemos la lista con la predicción de nuestro clasifica   
        return self.prediccion

In [116]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))


[array([[0.19083969, 0.46581197],
       [0.22900763, 0.33760684],
       [0.07442748, 0.04273504],
       [0.50572519, 0.15384615]])
 array([[0.62575639, 0.65001934],
       [0.37424361, 0.34998066]])
 array([[0.01904762, 0.07659574],
       [0.03428571, 0.1106383 ],
       [0.50666667, 0.5787234 ],
       [0.08952381, 0.0893617 ],
       [0.35047619, 0.14468085]])
 array([[0.21132075, 0.27916667],
       [0.12075472, 0.06666667],
       [0.01320755, 0.01666667],
       [0.16981132, 0.19166667],
       [0.31698113, 0.22083333],
       [0.01509434, 0.01666667],
       [0.01886792, 0.025     ],
       [0.04150943, 0.06666667],
       [0.01320755, 0.00833333],
       [0.07924528, 0.10833333]])
 array([[0.55114423, 0.52730337],
       [0.44885577, 0.47269663]])
 array([[0.56      , 0.71489362],
       [0.09333333, 0.11489362],
       [0.07619048, 0.04255319],
       [0.05142857, 0.0212766 ],
       [0.21904762, 0.10638298]])
 array([[0.05714286, 0.06382979],
       [0.14666667, 0.25957447

IndexError: index 48 is out of bounds for axis 0 with size 2

In [117]:
dataset = Datos('tic-tac-toe.data')
print(dataset.datos)

['TLeftSq', 'TMidSq', 'TRightSq', 'MLeftSq', 'MMidSq', 'MRightSq', 'BLeftSq', 'BMidSq', 'BRightSq', 'Class']
[{'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'negative': 0, 'positive': 1}]
[[2. 2. 2. ... 1. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 1. 2. 1.]
 ...
 [1. 2. 1. ... 1. 2. 0.]
 [1. 2. 1. ... 1. 2. 0.]
 [1. 1. 2. ... 2. 2. 0.]]


In [118]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)


VALIDACION SIMPLE Train: [520 790 761 917 182 681 250 857 242 679 522 145 364 272 119 610 393 493
  38 501 758 856 567 461 203 294 112 948 602 120 388 266 233 837 367 524
 248 901 763 269 336 184 260 680 733 401 786 669 653 862 625 816 302 757
 558 643 143 173 801 185 525 295 428  75  42 297 439 545  62 207 363 408
 792 712 264 590 708 343 308 205 125 164 375 430 832 725 382 196 953 839
 179 639 194 751 531 484 665 945 512 711 208 595 743 940 608 688   8 783
 802 654 765 813 921 793  86 627 689 114 285 911 325 247 499 624 136 815
 473 904 740 903  67 189 720 530 796 645 465 641 230  31 909 384  79 766
  69 280 842  73 445 674 519 906 323 154 559 198 952 592 887 937 744 224
 334 699 789 193 561  14 617 695 698 631 505 635 563 175 178 579 554 210
 715 283 831 436 803 453 900 666 569 724 340 865 485 255 897 177  59 440
 804 845 692 628 508 910  10 521  78 226 111 718 403 497 539 231 263 947
 823 642 310 713 618 144 259   4 702 406 212 110 709 600 891 599 155 550
 726 907 871 716 282 251 4

In [119]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

[array([[0.18650794, 0.2309322 ],
       [0.4484127 , 0.29661017],
       [0.36507937, 0.47245763]])
 array([[0.23809524, 0.29025424],
       [0.30555556, 0.375     ],
       [0.45634921, 0.33474576]])
 array([[0.18253968, 0.24788136],
       [0.45238095, 0.30720339],
       [0.36507937, 0.44491525]])
 array([[0.23412698, 0.27754237],
       [0.28968254, 0.35805085],
       [0.47619048, 0.36440678]])
 array([[0.14285714, 0.16101695],
       [0.57539683, 0.24576271],
       [0.28174603, 0.59322034]])
 array([[0.25      , 0.28601695],
       [0.32539683, 0.35169492],
       [0.42460317, 0.36228814]])
 array([[0.19444444, 0.22669492],
       [0.4484127 , 0.3029661 ],
       [0.35714286, 0.47033898]])
 array([[0.25793651, 0.28389831],
       [0.27380952, 0.36652542],
       [0.46825397, 0.34957627]])
 array([[0.20634921, 0.2309322 ],
       [0.41269841, 0.28177966],
       [0.38095238, 0.48728814]])]
[array([[0.20233463, 0.22269807],
       [0.43579767, 0.30835118],
       [0.3618677 , 0.4

In [120]:
dataset = Datos('german.data')
print(dataset.datos)
print(dataset.nominalAtributos)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'Class']
[{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, {}, {'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}, {'A40': 0, 'A41': 1, 'A410': 2, 'A42': 3, 'A43': 4, 'A44': 5, 'A45': 6, 'A46': 7, 'A48': 8, 'A49': 9}, {}, {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}, {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}, {}, {'A91': 0, 'A92': 1, 'A93': 2, 'A94': 3}, {'A101': 0, 'A102': 1, 'A103': 2}, {}, {'A121': 0, 'A122': 1, 'A123': 2, 'A124': 3}, {}, {'A141': 0, 'A142': 1, 'A143': 2}, {'A151': 0, 'A152': 1, 'A153': 2}, {}, {'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}, {}, {'A191': 0, 'A192': 1}, {'A201': 0, 'A202': 1}, {'1': 0, '2': 1}]
[[ 0.  6.  4. ...  1.  0.  0.]
 [ 1. 48.  2. ...  0.  0.  1.]
 [ 3. 12.  4. ...  0.  0.  0.]
 ...
 [ 3. 12.  2. ...  0.  0.  0.]
 [ 0. 45.  2. ...  1.  0.  1.]
 [ 1. 45.  4. ...  0.  0.  0.]]
[True, False, True, True, False,

In [121]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)

VALIDACION SIMPLE Train: [502 525  37 217 516 965 329 374 376 569 949 597 570   5 736 731 723 894
 183 236 700 855  26 489 343 201 575  70 408 585 291 465  86 670 459 637
 116 933 598 881 558 352   3 791 767 806 177 645  58 580 146 815 105 475
 684  97 388 760 797 451 192 646 606  89 544 954 500  79 989 437 152 278
 244 113 907   0 803 608 415 540 103 972 357 286 583 441 187 834 134 397
 662 510 538 215 230 749 321 828  94 986 141 810 922  80 622  77 928 340
 795 243 969 713 730 872 497 735 211 156 672 420 378  11 689 957 638 181
 883   8 798 660 462 285 207 800 276 313  38 847 171 661  67 801 263 657
 424  34 189 366  66 213 859 191 830 246 157 296 621 545 384 571 457 813
 618 522 964 365 368  63 229 394 382 118 160 979 763 197 601 879 746 556
 390  25 535 951   4 262  90 861 467 454 654 322 123 687 565 242 712 419
 783 863 870 326 180  42  96 317 316  10 876 182 991 936 929 941 707 305
 519 747 482 284 473 188 288 169 417 396 526 423 935 327 849 899 690 945
 766 145 605 914 464 513 1

In [122]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

[array([[0.18939394, 0.45217391],
       [0.24621212, 0.32173913],
       [0.06628788, 0.04347826],
       [0.49810606, 0.1826087 ]])
 array([[0.634436  , 0.65110325],
       [0.365564  , 0.34889675]])
 array([[0.02268431, 0.09090909],
       [0.03024575, 0.0995671 ],
       [0.51795841, 0.54112554],
       [0.0831758 , 0.1038961 ],
       [0.34593573, 0.16450216]])
 array([[0.20786517, 0.28389831],
       [0.11985019, 0.05508475],
       [0.00749064, 0.02542373],
       [0.17602996, 0.1779661 ],
       [0.31460674, 0.20762712],
       [0.01123596, 0.01694915],
       [0.02621723, 0.03389831],
       [0.04307116, 0.05932203],
       [0.01498127, 0.00847458],
       [0.07865169, 0.13135593]])
 array([[0.55564549, 0.52586983],
       [0.44435451, 0.47413017]])
 array([[0.54820416, 0.71428571],
       [0.10018904, 0.11688312],
       [0.06994329, 0.03896104],
       [0.05860113, 0.02164502],
       [0.22306238, 0.10822511]])
 array([[0.05293006, 0.0995671 ],
       [0.15122873, 0.24242424

IndexError: index 12 is out of bounds for axis 0 with size 2

In [123]:
#SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)



/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


Predicciones utilizando validacion simple sin correcion de Laplace: [0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 0.]
Clases reales de la particion de text [0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0

/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [87]:
# SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)


Predicciones utilizando validacion simple sin correcion de Laplace: [1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
Clases reales de la particion de text [1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 0

/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/home/aalcala/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha